In [42]:
import warnings

warnings.filterwarnings('ignore')

In [43]:
from pluto_survey_tools import QUESTIONNAIRE
from pluto_survey_tools.model import Question
import pluto_survey_tools.stats as stats
from IPython.display import display_markdown

## Min-max ranges

### Global

Overview of the possible minimum and maximum scores for each question and section of the questionnaire.

In [44]:
display_markdown(f'# Questionnaire: {stats.score_range_questionnaire(QUESTIONNAIRE)}', raw=True)

for section in QUESTIONNAIRE.sections:
    display_markdown(f'## {section.title}: {stats.score_range_section(section)}', raw=True)
    for i, question in enumerate(section.questions):
        display_markdown(f'Q{i+1}: {stats.score_range_question(question)}', raw=True)

# Questionnaire: (-60.0, 40.0)

## Information About the Applicant: (-8.0, 8.0)

Q1: (-1.0, 1.0)

Q2: (-2.0, 1.0)

Q3: (-1.0, 2.0)

...

Q3: (-3.0, 2.0)

Q4: (-1.0, 2.0)

Q5: (-1.0, 1.0)

Q6: (-3.0, 0.0)

Q7: (-3.0, 5.0)

### By axis

In [45]:
questions = QUESTIONNAIRE.questions
x_q = [q for q in questions if q.impact_keys[0] == 'x']
x_ranges = [stats.score_range_question(q) for q in x_q]
x_min = sum([r[0] for r in x_ranges])
x_max = sum([r[1] for r in x_ranges])

y_q = [q for q in questions if q.impact_keys[0] == 'y']
y_ranges = [stats.score_range_question(q) for q in y_q]
y_min = sum([r[0] for r in y_ranges])
y_max = sum([r[1] for r in y_ranges])

In [46]:
display_markdown(f'## X ~ Risk: ({x_min}, {x_max})', raw=True)
display_markdown(f'## Y ~ Public Value: ({y_min}, {y_max})', raw=True)

## X ~ Risk: (-51.0, 23.0)

## Y ~ Public Value: (-9.0, 17.0)

## Score distributions

In [47]:
import pandas as pd
from collections import Counter


def sums(n: int, nums: list[float | int]) -> list[float]:
    return [t[1] for t in stats.all_possible_sums_gen(n, nums)]


def merge_counters(c1: Counter, c2: Counter) -> Counter:
    c = Counter()
    for k1, v1 in c1.items():
        for k2, v2 in c2.items():
            c[k1 + k2] += v1 * v2
    return c


def score_count_freq(questions: list[Question]) -> Counter:
    counters = []
    for q in questions:
        scores = [c.score for c in q.choices]
        up_to_n = q.selection_range.end
        c = Counter(sums(up_to_n, scores))
        counters.append(c)

    c = counters[0]
    for c2 in counters[1:]:
        c = merge_counters(c, c2)
    return c


def score_count_df(counter: Counter) -> pd.DataFrame:
    df = pd.DataFrame({'score': list(counter.keys()), 'count': list(counter.values())})
    return df


def score_count_df_from_questions(questions: list[Question]) -> pd.DataFrame:
    c = score_count_freq(questions)
    df = score_count_df(c)
    return df


def normalize_df(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    df_copy = df.copy()
    df_copy[columns] -= df_copy[columns].min()
    df_copy[columns] /= df_copy[columns].max()
    return df_copy

In [48]:
import altair as alt


def hist(df: pd.DataFrame, title: str, titleX='Score', titleY='Count') -> alt.Chart:
    return alt.Chart(df).mark_bar().encode(
        x=alt.X('score', title=titleX),
        y=alt.Y('count', title=titleY),
        tooltip=['score', 'count']
    ).properties(
        title=title
    )

In [49]:
config = [
    ('All questions', questions),
    ('Risk', x_q),
    ('Public Value', y_q),
]
for title, questions in config:
    df = score_count_df_from_questions(questions)
    df_norm = normalize_df(df, ['count'])
    display(hist(df, title))
    display(hist(df_norm, f'{title} (normalized)', titleY='Normalized count'))

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)